# One-Hot-Encoding and Cross-Validation

In this module you are going to learn how to handle categorical features in a better way using one-hot-encoding. You will also learn how to use cross-validation to find a better estimate for the error of a model. You will also learn the difference between test sets and validation sets when comparing models.

<b>Functions and attributes in this lecture: </b>
- `sklearn.preprocessing` - Submodule for preprocessing
 - `OneHotEncoder` - One-hot-encoder for categorical data
- `sklearn.compose` - Submodule for composing transformers to put them into a pipeline
  - `make_column_transformer` - Transform only specific columns
- `sklearn.model_selection` - Submodule for choosing models
  - `cross_validate` - Cross validation on modules

In [1]:
# RUN THIS CELL!

# Non-sklearn packages
import numpy as np
import pandas as pd
from seaborn import load_dataset

# Import the tips dataset
tips = load_dataset("tips")
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


## One-Hot-Encoding

In [2]:
# Import the one-hot-encoder
from sklearn.preprocessing import OneHotEncoder

In [3]:
# Create a one-hot-encoder transformer
enc = OneHotEncoder()

In [4]:
# Fitting the data using the one-hot-encoder transformer
tra_array = enc.fit_transform(tips[['sex', 'smoker', 'day', 'time']])

In [5]:
# Seeing the categories in the one-hot-encoder transformer
enc.categories_

[array(['Female', 'Male'], dtype=object),
 array(['No', 'Yes'], dtype=object),
 array(['Fri', 'Sat', 'Sun', 'Thur'], dtype=object),
 array(['Dinner', 'Lunch'], dtype=object)]

In [6]:
# Creating a dataframe containing the new columns
transformed = pd.DataFrame(tra_array.toarray(), columns=enc.get_feature_names_out())
transformed.head()

,sex_Female,sex_Male,smoker_No,smoker_Yes,day_Fri,day_Sat,day_Sun,day_Thur,time_Dinner,time_Lunch
0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [7]:
# Adding the new columns to the tips dataframe
enc_tips = pd.concat([tips, transformed], axis=1)
enc_tips.head()

,total_bill,tip,sex,smoker,day,time,size,sex_Female,sex_Male,smoker_No,smoker_Yes,day_Fri,day_Sat,day_Sun,day_Thur,time_Dinner,time_Lunch
0,16.99,1.01,Female,No,Sun,Dinner,2,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,10.34,1.66,Male,No,Sun,Dinner,3,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,21.01,3.50,Male,No,Sun,Dinner,3,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,23.68,3.31,Male,No,Sun,Dinner,2,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,24.59,3.61,Female,No,Sun,Dinner,4,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [8]:
# Drop repeated columns
enc_tips.drop(['sex', 'smoker', 'day', 'time', 'sex_Male', 'smoker_Yes', 'time_Lunch'], 
             axis=1, inplace=True)

In [9]:
# Save the cleaned data for futher use
enc_tips.to_csv('cleaned_tips.csv', index=False)

## Cross-Validation

In [10]:
# Defining the features and targets
X = enc_tips.drop(['tip'], axis=1)
y = enc_tips[['tip']]

In [11]:
# Import cross_validation
from sklearn.model_selection import cross_validate

# Import linear regression
from sklearn.linear_model import LinearRegression

In [12]:
# Creating an instance of linear regression
lin_reg = LinearRegression()

In [13]:
# Doing cross validation with 5 folds and linear regression
result = cross_validate(lin_reg, X, y, cv=5, scoring='neg_mean_squared_error')

In [14]:
# The resulting scores
result['test_score']

array([-0.79379953, -0.65897166, -0.86707796, -1.90620955, -1.40034021])

In [15]:
# Mean of the scores
mean_test_score = -np.mean(result['test_score'])
mean_test_score

1.1252797795692773

## Using One-Hot-Encoding in a Pipeline

In [16]:
# Imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

In [17]:
# Defining the features and targets
X = tips.drop(['tip'], axis=1)
y = np.ravel(tips[['tip']])

In [18]:
# Splitting into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [19]:
# Importing the column transformer
from sklearn.compose import ColumnTransformer

In [20]:
# Creating a pipeline using column transformer
prepros_pipeline = ColumnTransformer([
    ('categorical', OneHotEncoder(), ['sex', 'smoker', 'day', 'time']),
    ('scaler', StandardScaler(), ['total_bill', 'size'])
    ])

In [21]:
# Setting two pipelines together
forest_pipeline = Pipeline([('preprossesing', prepros_pipeline),
                           ('RandomForest', RandomForestRegressor())])
linear_pipeline = Pipeline([('preprossesing', prepros_pipeline),
                           ('linear', LinearRegression())])

## Using a Pipeline in Cross-Validation

In [22]:
# Cross-validation can take in pipelines
result_forest = cross_validate(forest_pipeline, X_train, y_train, scoring='neg_mean_squared_error')
-np.mean(result_forest['test_score'])

1.2817414656003787

In [23]:
# cross-validation on linear regression
result_linear = cross_validate(linear_pipeline, X_train, y_train, scoring='neg_mean_squared_error')
-np.mean(result_linear['test_score'])

1.182481867863093

In [24]:
# We choose the linear regression since the error is lower. 
# Want to use the test set to find the error.
linear_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprossesing',
                 ColumnTransformer(transformers=[('categorical',
                                                  OneHotEncoder(),
                                                  ['sex', 'smoker', 'day',
                                                   'time']),
                                                 ('scaler', StandardScaler(),
                                                  ['total_bill', 'size'])])),
                ('linear', LinearRegression())])

In [25]:
# Predicting on the test set using the pipeline
y_pred = linear_pipeline.predict(X_test)

In [26]:
# Getting out the final error
final_error = mean_squared_error(y_pred, y_test)
final_error

0.9991539364049288

In [27]:
# Training on all data to get the final model used in production
final_model = linear_pipeline.fit(X, y)